In [2]:
## Prepare LGB for GPU

In [3]:
!rm -r /opt/conda/lib/python3.7/site-packages/lightgbm
!git clone --recursive https://github.com/Microsoft/LightGBM

Cloning into 'LightGBM'...
remote: Enumerating objects: 20455, done.
remote: Total 20455 (delta 0), reused 0 (delta 0), pack-reused 20455
Receiving objects: 100% (20455/20455), 16.07 MiB | 12.52 MiB/s, done.
Resolving deltas: 100% (14922/14922), done.
Submodule 'include/boost/compute' (https://github.com/boostorg/compute) registered for path 'compute'
Submodule 'external_libs/fast_double_parser' (https://github.com/lemire/fast_double_parser.git) registered for path 'external_libs/fast_double_parser'
Submodule 'external_libs/fmt' (https://github.com/fmtlib/fmt.git) registered for path 'external_libs/fmt'
Cloning into '/kaggle/working/LightGBM/compute'...
remote: Enumerating objects: 21728, done.        
remote: Total 21728 (delta 0), reused 0 (delta 0), pack-reused 21728        
Receiving objects: 100% (21728/21728), 8.51 MiB | 15.07 MiB/s, done.
Resolving deltas: 100% (17565/17565), done.
Cloning into '/kaggle/working/LightGBM/external_libs/fast_double_parser'...
remote: Enumerating ob

In [4]:
!apt-get install -y -qq libboost-all-dev

In [5]:
%%bash
cd LightGBM
rm -r build
mkdir build
cd build
cmake -DUSE_GPU=1 -DOpenCL_LIBRARY=/usr/local/cuda/lib64/libOpenCL.so -DOpenCL_INCLUDE_DIR=/usr/local/cuda/include/ ..
make -j$(nproc)

-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found OpenMP_C: -fopenmp (found version "4.5") 
-- Found OpenMP_CXX: -fopenmp (found version "4.5") 
-- Found OpenMP: TRUE (found version "4.5")  
-- Looking for CL_VERSION_2_2
-- Looking for CL_VERSION_2_2 - not found
-- Looking for CL_VERSION_2_1
-- Looking for CL_VERSION_2_1 - not found
-- Looking for CL_VERSION_2_0
-- Looking for CL_VERSION_2_0 - not found
-- Looking for CL_VERSION_1_2
-- Looking

rm: cannot remove 'build': No such file or directory
cmake: /opt/conda/lib/libcurl.so.4: no version information available (required by cmake)
/usr/bin/cmake: /opt/conda/lib/libcurl.so.4: no version information available (required by /usr/bin/cmake)
/usr/bin/cmake: /opt/conda/lib/libcurl.so.4: no version information available (required by /usr/bin/cmake)
/usr/bin/cmake: /opt/conda/lib/libcurl.so.4: no version information available (required by /usr/bin/cmake)
/usr/bin/cmake: /opt/conda/lib/libcurl.so.4: no version information available (required by /usr/bin/cmake)
/usr/bin/cmake: /opt/conda/lib/libcurl.so.4: no version information available (required by /usr/bin/cmake)
/usr/bin/cmake: /opt/conda/lib/libcurl.so.4: no version information available (required by /usr/bin/cmake)
/usr/bin/cmake: /opt/conda/lib/libcurl.so.4: no version information available (required by /usr/bin/cmake)
/usr/bin/cmake: /opt/conda/lib/libcurl.so.4: no version information available (required by /usr/bin/cmake)
/u

In [6]:
!cd LightGBM/python-package/;python3 setup.py install --precompile

running install
running build
running build_py
creating build
creating build/lib
creating build/lib/lightgbm
copying lightgbm/sklearn.py -> build/lib/lightgbm
copying lightgbm/compat.py -> build/lib/lightgbm
copying lightgbm/callback.py -> build/lib/lightgbm
copying lightgbm/engine.py -> build/lib/lightgbm
copying lightgbm/__init__.py -> build/lib/lightgbm
copying lightgbm/basic.py -> build/lib/lightgbm
copying lightgbm/libpath.py -> build/lib/lightgbm
copying lightgbm/plotting.py -> build/lib/lightgbm
running egg_info
creating lightgbm.egg-info
writing lightgbm.egg-info/PKG-INFO
writing dependency_links to lightgbm.egg-info/dependency_links.txt
writing requirements to lightgbm.egg-info/requires.txt
writing top-level names to lightgbm.egg-info/top_level.txt
writing manifest file 'lightgbm.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
no previously-included directories found matching 'build'
writing manifest file 'lightgbm.egg-info/SOURCES.txt'
copying lightgbm/VERSION.t

In [7]:
!mkdir -p /etc/OpenCL/vendors && echo "libnvidia-opencl.so.1" > /etc/OpenCL/vendors/nvidia.icd
!rm -r LightGBM

In [8]:
# Lib & Dependencies
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import requests
from scipy import stats

import gc
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from scipy.stats import rankdata

In [9]:
def spearman(y_true, y_pred):
    return (stats.spearmanr(y_true, y_pred))[0]

In [10]:
train_datalink = '/kaggle/input/datacrunch/train_test_hackathon.csv'
hackathon_data_link = '/kaggle/input/datacrunch/hackathon_data.csv'

# Data for training
train_df = pd.read_csv(train_datalink)

# Data for which you will submit your prediction
test_df = pd.read_csv(hackathon_data_link)

# Targets to be predicted
targets = train_df[train_df.columns[-3:]]

# Cleaning an ugly columns
train_df.drop(columns=['Unnamed: 0'], inplace = True)
test_df.drop(columns=['Unnamed: 0'], inplace = True)

In [11]:
# This is very special to Xgboost as it likes non-continuous targets.
d = {1:0.25, 4:1.0, 0:0.0, 2:0.50, 3:0.75}

label_map = {0.0: 0, 0.25: 1, 0.5: 2, 0.75: 3, 1.0: 4}
inv_label_map = {v:k for k,v in label_map.items()}

# for t in ['Target_1', 'Target_2', 'Target_3']:
#     targets[t] = targets[t].map(label_map)

In [12]:
orig_train_df = pd.read_csv(train_datalink)

features = [rf'Feature_{idx}' for idx in range(1, 251)]
df_all = pd.concat([train_df, test_df])

In [13]:
## Prepare aggregated features
# df_counts = df_all.groupby(['Moons'])[features].count().reset_index()
df_mean = df_all.groupby(['Moons'])[features].mean().reset_index()
df_std = df_all.groupby(['Moons'])[features].std().reset_index()

df_all = df_all.merge(df_mean, on='Moons', how='left', suffixes=("", "_mean")).merge(df_std, on='Moons', how='left', suffixes=("", "_std"))

for idx in range(1, 251):
    df_all[rf'Feature_{idx}_deviation'] = df_all[rf"Feature_{idx}"] - df_all[rf"Feature_{idx}_mean"]

for idx in range(1, 251):
    df_all[rf"Feature_{idx}_deviation_ratio"] = np.where(df_all[rf"Feature_{idx}_std"] == 0, 0, df_all[rf'Feature_{idx}_deviation'] / df_all[rf"Feature_{idx}_std"])

df_mean['Moons'] = df_mean['Moons'] + 1

df_all = df_all.merge(df_mean, on='Moons', how='left', suffixes=("", "_mean_minus_1"))

df_all = df_all[df_all.Feature_250_mean_minus_1.notnull()]

In [14]:
targets = targets.iloc[-df_all.shape[0]:]

In [15]:
df_all.shape, targets.shape

((119936, 1505), (70217, 3))

In [16]:
import gc
del df_mean, df_std
gc.collect()

84

In [17]:
df_all.drop(columns=['id', 'Moons'], inplace = True)
test_len = test_df.shape[0]
train_df = df_all.iloc[:-test_len]
test_df = df_all.iloc[-test_len:]
train_df.drop(columns=['Target_1', 'Target_2', 'Target_3'], inplace = True)
test_df.drop(columns=['Target_1', 'Target_2', 'Target_3'], inplace = True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [18]:
df_all.shape, train_df.shape, test_df.shape, targets.shape

((119936, 1503), (66095, 1500), (53841, 1500), (70217, 3))

# LGB for Target_2

In [19]:
def get_lgb_model():
    return lgb.LGBMRegressor(**{
    'learning_rate': 0.01,
#     'num_leaves': 31,
#     'max_bin': 1023,
#     'min_child_samples': 1000,
    'reg_alpha': 0.1,
    'reg_lambda': 0.2,
    'feature_fraction': 0.9,
#     'bagging_freq': 1,
    'bagging_fraction': 0.9,
#     'objective': 'multiclass',
    'n_jobs': -1,
    'n_estimators':120,
    'device': 'gpu',
    'gpu_platform_id': 0,
    'gpu_device_id': 0
#     'num_classes': 5
})

In [20]:
targets = targets.iloc[-train_df.shape[0]:]

In [21]:
train_df = train_df.reset_index(drop=True)
targets = targets.reset_index(drop=True)

In [22]:
orig_train_df = orig_train_df.iloc[-train_df.shape[0]:]

In [23]:
df_all.shape, train_df.shape, test_df.shape, targets.shape, orig_train_df.shape

((119936, 1503), (66095, 1500), (53841, 1500), (66095, 3), (66095, 256))

In [24]:
train_df = train_df.reset_index(drop=True)
orig_train_df = orig_train_df.reset_index(drop=True)

In [25]:
train_df.shape, test_df.shape

((66095, 1500), (53841, 1500))

In [26]:
prediction = pd.DataFrame()
results = []
for col in ['Target_2']:
    scores = []
    MODELS = []
    target = targets[col]
    skf = StratifiedKFold(n_splits=8, shuffle=True, random_state=11111)
    for fold_, (train_indexes, valid_indexes) in enumerate(skf.split(train_df, target.map(label_map))):
        y_pred = np.zeros(len(target)) 
        model = get_lgb_model()
        model.fit( train_df.loc[train_indexes], target[train_indexes],
                          eval_set = (train_df.loc[valid_indexes], target[valid_indexes]),
                          verbose = 40,
                          eval_metric='mse',
                          early_stopping_rounds=25)
        val_pred = model.predict(train_df.loc[valid_indexes])
        y_pred[valid_indexes] = val_pred
        score = spearman(target[valid_indexes], val_pred)
        scores.append(score)
        print(rf"Spearman score for Fold {fold_}: {score}")
        MODELS.append( model )
    preds = np.array([model.predict(test_df) for model in MODELS])
    prediction[col] = preds.mean(axis=0)
    results.append(rf"Average Spearman score for {col}: {np.mean(scores)}")

[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
Training until validation scores don't improve for 25 rounds
[40]	valid_0's l2: 0.11448
[80]	valid_0's l2: 0.11384
[120]	valid_0's l2: 0.113637
Did not meet early stopping. Best iteration is:
[120]	valid_0's l2: 0.113637
Spearman score for Fold 0: 0.14763660406787543
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
Training until validation scores don't improve for 25 rounds
[40]	valid_0's l2: 0.114381
[80]	valid_0's l2: 0.113707
[120]	valid_0's l2: 0.113464
Did not meet early stopping. Best iteration is:
[120]	valid_0's l2: 0.113464
Spearman score for Fold 1: 

# CNN for Target_1 and Target_3

In [27]:
scaler = StandardScaler()
train_data_scaled = scaler.fit_transform(train_df.values)
test_data = scaler.transform(test_df.values)

In [42]:
import keras

epochs = 3
learning_rate_init = 0.01
batch_size = 32

num_features = train_data_scaled.shape[1]

def get_model():
    inp = keras.layers.Input((num_features,))
    x = keras.layers.Reshape((num_features,1))(inp)
    x = keras.layers.Conv1D(128,1,strides=1, activation='elu')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Conv1D(32,1, activation='elu')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Conv1D(16,1, activation='elu')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Conv1D(4,1, activation='elu')(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Reshape((num_features*4,1))(x)
    x = keras.layers.AveragePooling1D(2)(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.BatchNormalization()(x)
    out = keras.layers.Dense(1)(x)
    return keras.Model(inputs=inp, outputs=out)


def lr_scheduler(epoch):
    if epoch <= epochs*0.8:
        return learning_rate_init
    else:
        return learning_rate_init * 0.1

def train_NN(data, target):
    optimizer = keras.optimizers.Adam(lr = learning_rate_init, decay = 0.00001)
    model = get_model()
    callbacks = []
    callbacks.append(keras.callbacks.LearningRateScheduler(lr_scheduler))
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    model.fit(data, target, epochs=epochs, verbose=1, batch_size=batch_size, callbacks=callbacks)
    return model

In [43]:
model = train_NN(train_data_scaled, targets['Target_1'])
prediction['Target_1'] = model.predict(test_data).ravel()
model = train_NN(train_data_scaled, targets['Target_3'])
prediction['Target_3'] = model.predict(test_data).ravel()

Epoch 1/3
2066/2066 [==============================] - 16s 8ms/step - loss: 0.7394
Epoch 2/3
2066/2066 [==============================] - 17s 8ms/step - loss: 0.1592
Epoch 3/3
2066/2066 [==============================] - 16s 8ms/step - loss: 0.1218
Epoch 1/3
2066/2066 [==============================] - 16s 8ms/step - loss: 0.7304
Epoch 2/3
2066/2066 [==============================] - 16s 8ms/step - loss: 0.1572
Epoch 3/3
2066/2066 [==============================] - 17s 8ms/step - loss: 0.1224


In [ ]:
prediction = prediction[['Target_1', 'Target_2', 'Target_3']]

In [ ]:
# prediction.to_csv('lgb_CNN2.csv', index=False)

In [ ]:
# API_KEY = "..." # <- HERE

# r = requests.post("https://hackathon.datacrunch.com/api/submission",
#     files = {
#         "file": ("x", prediction.to_csv().encode('ascii'))
#     },
#     data = {
#         "apiKey": API_KEY
#     },
# )

# if r.status_code == 200:
#     print("Submission submitted :)")
# elif r.status_code == 423:
#     print("ERR: Submissions are close")
#     print("The submissions are not enabled because the hackathon hasn't started yet or is already finished.")
#     print("Or the server is currently crunching the submitted files, please wait some time before retrying.")
# elif r.status_code == 422:
#     print("ERR: API Key is missing or empty")
#     print("Did you forget to fill the API_KEY variable?")
# elif r.status_code == 404:
#     print("ERR: Unknown API Key")
#     print("You should check that the provided API key is valid and is the same as the one you've received by email.")
# elif r.status_code == 400:
#     print("ERR: The file must not be empty")
#     print("You have send a empty file.")
# elif r.status_code == 401:
#     print("ERR: Your email hasn't been verified")
#     print("Please verify your email or contact a cruncher.")
# elif r.status_code == 429:
#     print("ERR: Too many submissions")
# else:
#     print("ERR: Server returned: " + str(r.status_code))
#     print("Ouch! It seems that we were not expecting this kind of result from the server, if the probleme persist, contact a cruncher.")

# IGNORE

In [ ]:
# df_all[['Moons', 'Feature_189']].groupby('Moons')['Feature_189'].mean().plot()
# df_all[['Moons', 'Feature_50']].groupby('Moons')['Feature_50'].mean().plot()
# feat50_dict = df_all[['Moons', 'Feature_50']].groupby('Moons')['Feature_50'].mean().to_dict()

In [ ]:
# # Check Correlations
# corrs = {}
# for feat in [rf"Feature_{idx}" for idx in range(1, 251)]:
#     corrs[feat] = np.corrcoef(train_df['Target_1'], train_df[feat] )[1][0]

# sorted(corrs.items(),key=lambda x: -x[1])

# # Check Correlations
# corrs = {}
# for feat in [rf"Feature_{idx}" for idx in range(1, 251)]:
#     corrs[feat] = np.corrcoef(train_df['Target_3'], train_df[feat] )[1][0]

# sorted(corrs.items(),key=lambda x: -x[1])

In [ ]:
# train_df = train_df.groupby('Moons').mean().reset_index()

# df_all = df_all.drop('id', axis=1)

# all_df_agg = df_all.groupby('Moons').mean().reset_index()

# all_df_agg['Feature_58'].plot()

# feat58_dict = pd.Series(all_df_agg.Feature_58.values, index=all_df_agg.Moons).to_dict()

# feat94_dict = pd.Series(all_df_agg.Feature_94.values, index=all_df_agg.Moons).to_dict()

# test_df['feat58'] = test_df.Moons.map(feat58_dict)

# test_df['feat94'] = test_df.Moons.map(feat94_dict)